### Applying the stride split to all markers (putting it all together)
So far we only worked with the Poll marker to look at data. Now lets apply the data imputation, filter and stride split on all of our markers. Beacuse the filter will not work on non-finite data, we use the gapfill without thinking about the maximum fill length. Once the filtering is done we will put back the nan values. Once the data has been filtered we will apply the stride split to look at data stride by stride. 
